In [4]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from statsmodels.stats import proportion as proptests

import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
data = pd.read_csv('homepage-experiment-data.csv')
data.head(10)

,Day,Control Cookies,Control Downloads,Control Licenses,Experiment Cookies,Experiment Downloads,Experiment Licenses
0,1,1764,246,1,1850,339,3
1,2,1541,234,2,1590,281,2
2,3,1457,240,1,1515,274,1
3,4,1587,224,1,1541,284,2
4,5,1606,253,2,1643,292,3
5,6,1681,287,3,1780,299,3
6,7,1534,262,5,1555,276,8
7,8,1798,331,12,1787,326,20
8,9,1478,223,30,1553,298,38
9,10,1461,236,32,1458,289,23


In [6]:
data.shape[0]

29

### Invariant metric

In [50]:
"""
H0: even distribution in control and experiment group
H1: the distribution is unequal in control and experiment group

Type 1 error rate: 0.05
"""

n_control = data['Control Cookies'].sum()
n_experiment = data['Experiment Cookies'].sum()
n_obs = n_control + n_experiment

print("Total number of observations: ", n_obs)
print("Total number of observations in the control group: ", n_control)
print("Total number of observations in the experiment group: ", n_experiment)

# probability of user belonging to either group
p_null = 0.5

sd = np.sqrt(p_null * (1 - p_null) * n_obs)
mean = p_null * n_obs

# P(X >= n_experiment)
exp_correction = n_experiment - 0.5

# P(X <= n_control)
control_correction = n_control + 0.5

z_exp = (exp_correction - mean) / sd
z_control = (control_correction - mean) / sd

print(stats.norm.cdf(z_control) + 1 - stats.norm.cdf(z_exp))

Total number of observations:  94197
Total number of observations in the control group:  46851
Total number of observations in the experiment group:  47346
0.10749294050130409


Since, the p-value is greater than 0.05 type 1 error rate, we fail to reject H0, i.e, the distribution is almost even

### Evaluation metrics

In [71]:
# downloads
"""
H0: experiment downloads <= control downloads
H1: experiment downloads > control downloads
"""
n_control_downloads = data['Control Downloads'].sum()
n_experiment_downloads = data['Experiment Downloads'].sum()

# control group download rate = base probability based on historic data = p_null
mean_control_downloads = n_control_downloads / n_control

# experiment group download rate
mean_experiment_downloads = n_experiment_downloads / n_experiment

# observation download rate
p_obs_downloads = (n_control_downloads + n_experiment_downloads) / (n_control + n_experiment)

# assuming one trial
var_obs = p_obs_downloads * (1 - p_obs_downloads) * 1

sd = np.sqrt(var_obs * (1 / n_control + 1 / n_experiment))

# two-sampled z-test
z = (mean_experiment_downloads - mean_control_downloads - 0) / sd
print(z)
print(1 - stats.norm.cdf(z))

7.870833726066236
1.7763568394002505e-15


In [74]:
# license
"""
H0: experiment purchases <= control purchases
H1: experiment purchases > control purchases
"""
n_control = data.query('Day < 22')['Control Cookies'].sum()
n_experiment = data.query('Day < 22')['Experiment Cookies'].sum()

n_control_licenses = data['Control Licenses'].sum()
n_experiment_licenses = data['Experiment Licenses'].sum()

# control group licenses rate = base probability based on historic data = p_null
mean_control_licenses = n_control_licenses / n_control

# experiment group licenses rate
mean_experiment_licenses = n_experiment_licenses / n_experiment

# observation licenses rate
p_obs_licenses = (n_control_licenses + n_experiment_licenses) / (n_control + n_experiment)

# assuming one trial
var_obs = p_obs_licenses * (1 - p_obs_licenses) * 1

sd = np.sqrt(var_obs * (1 / n_control + 1 / n_experiment))

# two-sampled z-test
z = (mean_experiment_licenses - mean_control_licenses - 0) / sd
print(z)
print(1 - stats.norm.cdf(z))

0.2586750111658684
0.3979430008399871


The statistical significance wasn't obtained for the number of licenses purchased, but the new homepage appeared to have a strong effect on the number of downloads made. So, this feature can be deployes as we initially considered, if only one of our metrics shows a statistically significant positive change we should be happy enough to deploy the new homepage.